## Day 76 Lecture 2 Assignment

In this assignment, we will learn about Dask dataframes. We will use the census dataset loaded below and perform some exploratory data analysis.

In [1]:
import dask.dataframe as dd
import numpy as np

In [52]:
census = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Big%20Data/acs2015_census_tract_data.csv')

In [3]:
census.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,...,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,...,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2


The dataset described above contains the aggregated data for each county in the United States. For example, the number in the Carpool column describes the percent of people in the county who carpool to work.

Print all the column names in the dataset and describe the data in each column

In [29]:
#Answer below:
census

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,int64,int64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


How many counties have a unemployement rate above 10%?

In [17]:
#Answer below:
unem = census[census['Unemployment'] > 10].compute()

In [22]:
unem.County.value_counts().count()

1423

Print the number of counties in each state.

In [72]:
# Answer below:
census.groupby(['State'])['County'].nunique().compute()

State
Alabama                  67
Alaska                   29
Arizona                  15
Arkansas                 75
California               58
Colorado                 64
Connecticut               8
Delaware                  3
District of Columbia      1
Florida                  67
Georgia                 159
Hawaii                    5
Idaho                    44
Illinois                102
Indiana                  92
Iowa                     99
Kansas                  105
Kentucky                120
Louisiana                64
Maine                    16
Maryland                 24
Massachusetts            14
Michigan                 83
Minnesota                87
Mississippi              82
Missouri                115
Montana                  56
Nebraska                 93
Nevada                   17
New Hampshire            10
New Jersey               21
New Mexico               33
New York                 62
North Carolina          100
North Dakota             53
Ohio          

Add a high child poverty boolean column. This column will be 1 if the rate of child poverty is over %20 and 0 otherwise.

In [24]:
census['ChildPoverty'].compute()

0         8.4
1        40.3
2        19.7
3         1.6
4        17.5
         ... 
73996    59.9
73997    28.0
73998    83.5
73999    34.6
74000    75.3
Name: ChildPoverty, Length: 74001, dtype: float64

In [53]:
# Answer below:
census['child_pov'] = census['ChildPoverty'].map(lambda x: 1 if x > 20 else 0)


In [54]:
census

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,child_pov
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,int64,int64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


What are the top 10 most populated counties?

In [68]:
#answer below:
a= census.groupby(['State','County'])['TotalPop'].sum().compute()

In [69]:
top10 = a.sort_values(ascending=False).head(10)

In [70]:
top10

State       County     
California  Los Angeles    10038388
Illinois    Cook            5236393
Texas       Harris          4356362
Arizona     Maricopa        4018143
California  San Diego       3223096
            Orange          3116069
Florida     Miami-Dade      2639042
New York    Kings           2595259
Texas       Dallas          2485003
New York    Queens          2301139
Name: TotalPop, dtype: int64

What is the current number of partitions? 

In [71]:
#Answer below:
census.npartitions


1